In [3]:
from sklearn.base import BaseEstimator,TransformerMixin,RegressorMixin,clone
from sklearn.model_selection import KFold,cross_val_score,train_test_split
import numpy as np

- Blend models

In [6]:
class BlendingAverageModels(BaseEstimator,RegressorMixin,TransformerMixin):
    def __init__(self,base_models,meta_model,holdout_pct=0.2,use_features_in_secondary=False):
        self.base_models=base_models
        self.meta_model=meta_model
        self.holdout_pct=holdout_pct
        self.use_features_in_secondary=use_features_in_secondary
    
    def fit(self,X,y):
        self.base_models_=[clone(x) for x in self.base_models]
        self.meta_model_=clone(self.meta_model)
        X_train,X_holdout,y_train,y_holdout=train_test_split(X,y,test_size=self.holdout_pct)
        holdout_predicitons=np.zeros((X_holdout.shape[0],len(self.base_models)))
        for i ,model in enumerate(self.base_models_):
            model.fit(X_train,y_train)
            y_pred=model.predict(X_holdout)
            holdout_predicitons[:,i]=y_pred
        if self.use_features_in_secondary:
            self.meta_model_.fit(np.hstack((X_holdout,holdout_predicitons)),y_holdout)
        else:
            self.meta_model_.fit(holdout_predicitons,y_holdout)
        
        return self
        
    def predict(self,X):
        meta_features=np.column_stack([model.predict(X) for model in self.base_models_])
        if self.use_features_in_secondary:
            return self.meta_model_.predict(np.hstack((X,meta_features)))
        else:
            return self.meta_model_.predict(meta_model)

In [8]:
def StackingAverageModels(BaseEstimator,TransformerMixin,RegressorMixin):
    def __int__(self,base_models,meta_model,n_folds=5,use_features_in_secondary=False):
        self.base_models=base_models
        self.meta_model=meta_model
        self.n_folds=n_folds
        self.use_features_in_secondary=use_features_in_secondary
    
    def fit(self,X,y):
        self.base_models_=[list() for x in self.base_models]
        self.meta_model_=clone(self.meta_model)
        kfold=KFold(n_splits=self.n_folds,shuffle=True,random_state=42)
        
        out_of_fold_predictions=np.zeros((X.shape[0],len(self.base_models)))
        for i ,model in enumerate(self.base_models):
            for train_index,holdout_index in kfold.split(X,y):
                instance=clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index],y[train_index])
                y_pred=instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index,i]=y_pred
        if self.use_features_in_secondary:
            self.meta_model_.fit(np.hstack((X,out_of_fold_predictions)),y)
        else:
            self.meta_model_.fit(out_of_fold_predictions,y)
        return self
    
    def predict(self,X):
        meta_features=np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1) 
            for base_models in self.base_models_])
        if self.use_features_in_secondary :
            return self.meta_model_.predict(np.hstack((X,meta_features)))
        else :
            return self.meta_model_.predict(X,meta_features)
        